In [1]:
import pandas as pd

In [2]:
import os
print(os.getcwd())


C:\Users\54115\Desktop\Python


In [3]:
ruta_dta = 'C:\\Users\\54115\\Desktop\\Python\\TP2\\usu_individual_T104.dta'
df_dta = pd.read_stata(ruta_dta)
print(df_dta.head())

     CODUSU  nro_hogar componente h15    ano4       trimestre  \
0  125098          1.0        1.0  Sí  2004.0  1er. Trimestre   
1  125397          1.0        1.0  Sí  2004.0  1er. Trimestre   
2  125515          1.0        1.0  Sí  2004.0  1er. Trimestre   
3  125558          1.0        1.0  Sí  2004.0  1er. Trimestre   
4  125558          1.0        2.0  Sí  2004.0  1er. Trimestre   

              region mas_500              aglomerado  pondera  ... deccfr  \
0  Gran Buenos Aires       S  Ciudad de Buenos Aires   1299.0  ...     07   
1  Gran Buenos Aires       S  Ciudad de Buenos Aires   2018.0  ...     00   
2  Gran Buenos Aires       S  Ciudad de Buenos Aires   1486.0  ...     07   
3  Gran Buenos Aires       S  Ciudad de Buenos Aires   1674.0  ...     08   
4  Gran Buenos Aires       S  Ciudad de Buenos Aires   1674.0  ...     08   

  ideccfr rdeccfr gdeccfr pdeccfr adeccfr pj1_1 pj2_1 pj3_1 idimpp  
0              06      06              04   0.0   0.0   0.0  00000  
1       

In [4]:
ruta_xls = 'C:\\Users\\54115\\Desktop\\Python\\TP2\\usu_individual_T124.xlsx'
df_xls = pd.read_excel(ruta_xls, engine="openpyxl")
print(df_xls.head())

                          CODUSU  ANO4  TRIMESTRE  NRO_HOGAR  COMPONENTE  H15  \
0  TQRMNOPUTHLMKQCDEGGFB00852588  2024          1          1           3    1   
1  TQRMNOPUTHLMKQCDEGGFB00852588  2024          1          1           4    1   
2  TQRMNOPUTHLMKQCDEGGFB00852588  2024          1          1           5    0   
3  TQRMNOPPWHLLKRCDEGGFB00852574  2024          1          1           1    1   
4  TQRMNOPPWHLLKRCDEGGFB00852574  2024          1          1           2    1   

   REGION MAS_500  AGLOMERADO  PONDERA  ...  PDECIFR  ADECIFR      IPCF  \
0      42       S          10      439  ...      NaN       12       0.0   
1      42       S          10      439  ...      NaN       12       0.0   
2      42       S          10      439  ...      NaN       12       0.0   
3      42       S          10      675  ...      NaN        8  212500.0   
4      42       S          10      675  ...      NaN        8  212500.0   

   DECCFR  IDECCFR  RDECCFR  GDECCFR  PDECCFR  ADECCFR  PONDIH

In [5]:
eph_04 = pd.read_stata(ruta_dta)
eph_24 = pd.read_excel(ruta_xls, engine='openpyxl')

eph_24.columns = eph_24.columns.str.lower()

In [6]:
gba_04 = eph_04[eph_04["region"] == 1].copy()
gba_24 = eph_24[eph_24["region"] == 1].copy()

eph_gba = pd.concat([gba_04, gba_24], ignore_index=True)

C:\Users\54115\AppData\Local\Temp\ipykernel_19120\136780333.py:4: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  eph_gba = pd.concat([gba_04, gba_24], ignore_index=True)


In [8]:
# Cambiamos el nombre de ciertas variables para facilitar el análisis
eph_gba = eph_gba.rename(columns={
    'ch04': 'genero',
    'ch06': 'edad',
    'ch07': 'estado_civil',
    'ch08': 'cobertura_medica',
    'nivel_ed': 'nivel_educativo',
    'cat_inac': 'categoria_inactividad',
    'ipcf': 'ingreso_per_capita_familiar',
})

In [10]:
variab_interes = [
    'genero', 'edad', 'estado_civil', 'cobertura_medica', 'nivel_educativo', 'estado',
    'categoria_inactividad', 'ingreso_per_capita_familiar', 'ano4', 
]

eph_gba_selected = eph_gba[variab_interes]
na_count = eph_gba_selected.groupby('ano4')[variab_interes].apply(lambda x: x.isna().sum())
print(na_count)

        genero  edad  estado_civil  cobertura_medica  nivel_educativo  estado  \
ano4                                                                            
2024.0       0     0             0                 0                0       0   

        categoria_inactividad  ingreso_per_capita_familiar  ano4  
ano4                                                              
2024.0                      0                            0     0  


In [12]:
print(eph_gba_selected['cobertura_medica'].dropna())

0       1
1       1
2       1
3       1
4       1
       ..
7046    1
7047    1
7048    1
7049    1
7050    1
Name: cobertura_medica, Length: 7051, dtype: object


In [14]:
mapa_cobertura = {
    'Obra social (incluye PAMI)': 1,
    'Mutual/Prepaga/Servicio de emergencia': 2,
    'Planes y seguros públicos': 3,
    'No paga ni le descuentan': 4,
    'Obra social y mutual/prepaga/servicio de emergencia': 12,
    'Obra social y planes y seguros públicos': 13,
    'Mutual/prepaga/servicio de emergencia/planes y seguros públi': 23,
    'Obra social, mutual/prepaga/servicio de emergencia y planes': 123
}

eph_gba_selected = eph_gba_selected.copy()
eph_gba_selected['cobertura_medica_num'] = eph_gba_selected['cobertura_medica'].map(mapa_cobertura)
print(eph_gba_selected[['cobertura_medica', 'cobertura_medica_num']].head(25))

   cobertura_medica  cobertura_medica_num
0                 1                   NaN
1                 1                   NaN
2                 1                   NaN
3                 1                   NaN
4                 1                   NaN
5                 1                   NaN
6                 1                   NaN
7                 1                   NaN
8                 4                   NaN
9                 1                   NaN
10                1                   NaN
11                4                   NaN
12                1                   NaN
13                4                   NaN
14                4                   NaN
15                4                   NaN
16                4                   NaN
17                1                   NaN
18                1                   NaN
19                1                   NaN
20                1                   NaN
21                1                   NaN
22                1               

In [19]:
print(eph_gba_selected.head(20))

   genero edad estado_civil cobertura_medica nivel_educativo estado  \
0       2   85            4                1               2      1   
1       1   59            2                1               6      1   
2       2   57            2                1               6      1   
3       1   26            5                1               6      1   
4       2   88            4                1               1      3   
5       2   81            3                1               2      1   
6       1   43            3                1               4      1   
7       1   66            3                1               6      1   
8       2   50            5                4               4      1   
9       1   30            5                1               3      1   
10      2   30            3                1               4      1   
11      1   10            5                4               1      3   
12      1    5            5                1               7      4   
13    